<a href="https://colab.research.google.com/github/satria-mitra/CASA0018-Cloud-Classification/blob/main/Projects/Final_Project/CASA0018_cloud_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Instruction**

This project, Cloud classification, is part of CASA0018 Deep Learning project. The dataset is available to download from the link below. However, some images are taken by myself or from publis image repository such as Flickr. Follow the steps on this tutorial to see the result. One may have different result since the datasets are not identic.


1.   Download Cirrus Cumulus Stratus Nimbus (CCSN) Database
https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/CADDPD&version=2.0

2.   Download Howard-Cloud-X
https://www.kaggle.com/datasets/imbikramsaha/howard-cloudx/code

## **Data Cleaning**
#### Manual checking the image

While CCSN datasets has 10 cloud types + 1 contrail, Howard-Cloud-X only has 10 cloud types without contrail.

First, lets see how many pictures for each type of cloud from the two datasets

In [2]:
import cv2 #opencv
import os
import time
import uuid

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
CCSN_path = '/content/drive/MyDrive/Kuliah/UCL/Projects/CASA0018-Cloud-Classification/Projects/Final_Project/Datasets/CCSN_v2' #Directories of Google Drive

os.chdir(CCSN_path) #Go to path
os.listdir(CCSN_path) #List out files in the path
print("Files and directories in '", CCSN_path, "' :", os.listdir(CCSN_path))

Files and directories in ' /content/drive/MyDrive/Kuliah/UCL/Projects/CASA0018-Cloud-Classification/Projects/Final_Project/Datasets/CCSN_v2 ' : ['Sc', 'Ns', 'Ci', 'Cu', 'Cs', 'Ct', 'St', 'As', 'Cc', 'Ac', 'Cb']


In [5]:
Howard_path = '/content/drive/MyDrive/Kuliah/UCL/Projects/CASA0018-Cloud-Classification/Projects/Final_Project/Datasets/Howard-Cloud-X' #Directories of Google Drive
os.chdir(Howard_path) #Go to path
os.listdir(Howard_path) #List out files in the path
print("Files and directories in '", Howard_path, "' :", os.listdir(Howard_path))

Files and directories in ' /content/drive/MyDrive/Kuliah/UCL/Projects/CASA0018-Cloud-Classification/Projects/Final_Project/Datasets/Howard-Cloud-X ' : ['Ac', 'As', 'Cc', 'Cs', 'Ci', 'Cb', 'Cu', 'Ns', 'Sc', 'St']


In [6]:
CCSN_dir = os.listdir(CCSN_path)
for dir_name in CCSN_dir:
    # Create the full path to the directory
    dir_path = os.path.join(CCSN_path, dir_name)

    # Get a list of files in the directory
    files = os.listdir(dir_path)

    # Count the number of files
    num_files = len(files)

    # Print the directory name and the number of files it contains
    print(f"{dir_name}: {num_files} files")

Sc: 208 files
Ns: 162 files
Ci: 74 files
Cu: 109 files
Cs: 58 files
Ct: 155 files
St: 202 files
As: 65 files
Cc: 105 files
Ac: 118 files
Cb: 193 files


In [7]:
Howard_dir = os.listdir(Howard_path)
for dir_name in Howard_dir:
    # Create the full path to the directory
    dir_path = os.path.join(Howard_path, dir_name)

    # Get a list of files in the directory
    files = os.listdir(dir_path)

    # Count the number of files
    num_files = len(files)

    # Print the directory name and the number of files it contains
    print(f"{dir_name}: {num_files} files")

Ac: 136 files
As: 187 files
Cc: 131 files
Cs: 129 files
Ci: 120 files
Cb: 124 files
Cu: 188 files
Ns: 131 files
Sc: 132 files
St: 137 files


#### Combine files from both datasets

In [15]:
import shutil

combined_path = '/content/combined_datasets'
for dir_name in os.listdir(CCSN_path):
    combined_subdir = os.path.join(combined_path, dir_name)
    if not os.path.exists(combined_subdir):
        os.makedirs(combined_subdir)

# Function to copy files from source to destination
def copy_files(source_path):
    for dir_name in os.listdir(source_path):
        source_dir_path = os.path.join(source_path, dir_name)
        destination_dir_path = os.path.join(combined_path, dir_name)

        if not os.path.exists(destination_dir_path):
            os.makedirs(destination_dir_path)


        for file_name in os.listdir(source_dir_path):
            src_file_path = os.path.join(source_dir_path, file_name)
            dst_file_path = os.path.join(destination_dir_path, file_name)

            file_counter = 1
            while os.path.exists(dst_file_path):
                name, ext = os.path.splitext(file_name)
                new_name = f"{name}_{file_counter}{ext}"
                dst_file_path = os.path.join(destination_dir_path, new_name)
                file_counter += 1

            # Copy the file to the destination directory
            shutil.copy(src_file_path, dst_file_path)

copy_files(CCSN_path)
copy_files(Howard_path)


#### Recheck how many files there are in each cloud types

In [21]:
combined_path = '/content/combined_datasets'
combined_dir = os.listdir(combined_path)

for dir_name in combined_dir:
    # Create the full path to the directory
    dir_path = os.path.join(combined_path, dir_name)

    # Get a list of files in the directory
    files = os.listdir(dir_path)

    # Count the number of files
    num_files = len(files)

    # Print the directory name and the number of files it contains
    print(f"{dir_name}: {num_files} files")

Ci: 194 files
Ns: 293 files
Sc: 340 files
Cu: 297 files
Cc: 236 files
Ac: 254 files
As: 252 files
St: 339 files
Cb: 317 files
Cs: 187 files
Ct: 155 files


## **Data Processing**
#### Import Dependancies and modules required

In [ ]:
import os #for creating path names and manipulating directories/files in an operating system
import random
import tensorflow as tf

In [ ]:
import os #for creating path names and manipulating directories/files in an operating system
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from PIL import Image
import PIL
import numpy as np
import cv2
import matplotlib.pyplot as plt
import datetime #for tensorboard and logging

print(tf.__version__)

2.15.0


### Connect to Data Directory in Google Drive

##### CCSN Datasets

##### Howard Datasets